In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("sierra-stem-firebase-adminsdk-o770a-3d7617ac20.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [15]:
import pandas as pd
import numpy as np

In [7]:
# Note: Use of CollectionRef stream() is prefered to get()
docs = (
    db.collection("test-trip")
#     .where(filter=FieldFilter("capital", "==", True))
    .stream()
)

# converts the document items into list of lists and then turns it into a dataframe
items = list(map(lambda x: {**x.to_dict(), 'id': x.id}, docs))
df = pd.DataFrame(items)

# set id as automatically generated index 
df.set_index('id', inplace=True)

# orders the dataframe columns by question 
df = df[[
    'firstName', 'lastName', 'dob', 'pronouns', 'pronounsDetails', 'heightFeet',
    'heightInches', 'weight', 'insurance', 'physicalActivityConcerns', 'allergies',
    'allergiesDetails', 'dietaryRestrictions', 'dietaryRestrictionsDetails',
    'otherAllergies', 'otherAllergiesDetails', 'medications', 'medicationsDetails',
    'psychConditions', 'psychConditionsDetails', 'healthDisabilities',
    'healthDisabilitiesDetails', 'medicalConditions', 'medicalConditionsDetails',
    'otherConcerns', 'ECFullName', 'ECRelationship', 'ECNumber', 'ECEmail', 'consentName'
]]

df

,firstName,lastName,dob,pronouns,pronounsDetails,heightFeet,heightInches,weight,insurance,physicalActivityConcerns,...,otherAllergiesDetails,medications,medicationsDetails,psychConditions,psychConditionsDetails,healthDisabilities,healthDisabilitiesDetails,medicalConditions,medicalConditionsDetails,otherConcerns
id,,,,,,,,,,,,,,,,,,,,,
3N7cAZgcP2mPY4I1LeGn,Alex,Baumgartner,2003-11-16,he/him,,6,0,180,State Farm,very fast i am!,...,pine bark extract.,no,{},yes,gaseous.,no,,[],,
94uaRdZ9gUbrEuuBpqg3,Alina,Chen,2003-09-14,she/her,,12,13,120,Kaiser Permanente,,...,,no,,no,,no,,[],,
L1rgGwC11HM8IuwTOdIY,Jane,Doe,1111-11-11,she/her,,5,10,190,kaiser,,...,,yes,"[{'medName': 'b', 'selfAdmin': 'b', 'dosageFre...",no,,no,,[],,
XzkoIkEs6440B0zCB17G,John,Doe,1111-11-11,she/her,,12,10,180,statefarm,,...,,yes,{},no,,no,,[],,
haItGXi74WmDJ9rKv8Cu,Alina,Chen,2004-09-14,she/her,,12,12,180,Kaiser Permanents,,...,,yes,{},no,,no,,[],,
k0Dlvf2amfTUM3ejugnJ,Alina,Chen,1111-11-11,she/her,,12,12,100,statefarm,,...,,yes,"[{'medName': 'a', 'selfAdmin': 'a', 'dosageFre...",no,,no,,[],,
vp9L1YxDmkx8rSYOw2q6,Alina,Chen,2000-10-10,she/her,,12,12,130,Kaiser Permanente,,...,,no,,no,,no,,[],,


In [18]:
def remove_empty_dicts(obj):
    if isinstance(obj, dict) and not obj or not obj:
        return np.nan
    return obj

# remove empty dictionaries from medications details
df['medicationsDetails'] = df['medicationsDetails'].apply(remove_empty_dicts)

# replace empty strings and arrays with None
df.replace('', np.nan, inplace=True)
df.replace([], np.nan, inplace=True)
df

,firstName,lastName,dob,pronouns,pronounsDetails,heightFeet,heightInches,weight,insurance,physicalActivityConcerns,...,otherAllergiesDetails,medications,medicationsDetails,psychConditions,psychConditionsDetails,healthDisabilities,healthDisabilitiesDetails,medicalConditions,medicalConditionsDetails,otherConcerns
id,,,,,,,,,,,,,,,,,,,,,
3N7cAZgcP2mPY4I1LeGn,Alex,Baumgartner,2003-11-16,he/him,NaN,6,0,180,State Farm,very fast i am!,...,pine bark extract.,no,NaN,yes,gaseous.,no,NaN,[],NaN,NaN
94uaRdZ9gUbrEuuBpqg3,Alina,Chen,2003-09-14,she/her,NaN,12,13,120,Kaiser Permanente,NaN,...,NaN,no,NaN,no,NaN,no,NaN,[],NaN,NaN
L1rgGwC11HM8IuwTOdIY,Jane,Doe,1111-11-11,she/her,NaN,5,10,190,kaiser,NaN,...,NaN,yes,"[{'medName': 'b', 'selfAdmin': 'b', 'dosageFre...",no,NaN,no,NaN,[],NaN,NaN
XzkoIkEs6440B0zCB17G,John,Doe,1111-11-11,she/her,NaN,12,10,180,statefarm,NaN,...,NaN,yes,NaN,no,NaN,no,NaN,[],NaN,NaN
haItGXi74WmDJ9rKv8Cu,Alina,Chen,2004-09-14,she/her,NaN,12,12,180,Kaiser Permanents,NaN,...,NaN,yes,NaN,no,NaN,no,NaN,[],NaN,NaN
k0Dlvf2amfTUM3ejugnJ,Alina,Chen,1111-11-11,she/her,NaN,12,12,100,statefarm,NaN,...,NaN,yes,"[{'medName': 'a', 'selfAdmin': 'a', 'dosageFre...",no,NaN,no,NaN,[],NaN,NaN
vp9L1YxDmkx8rSYOw2q6,Alina,Chen,2000-10-10,she/her,NaN,12,12,130,Kaiser Permanente,NaN,...,NaN,no,NaN,no,NaN,no,NaN,[],NaN,NaN


In [5]:
# syntax for grabbing multiple columns into a new dataframe 
name_allergies = df[['firstName', 'lastName', 'allergies','allergiesDetails']]
name_allergies

,firstName,lastName,allergies,allergiesDetails
id,,,,
3N7cAZgcP2mPY4I1LeGn,Alex,Baumgartner,no,
94uaRdZ9gUbrEuuBpqg3,Alina,Chen,no,
T35fHnKDGpzhIzU4XsFZ,Alina,Chen,no,
vp9L1YxDmkx8rSYOw2q6,Alina,Chen,yes,"peanuts, grass"


In [90]:
# syntax for grabbing multiple columns into a new dataframe 
new_df = df.loc[df['medicationsDetails'].notnull(), ['firstName', 'lastName','medicationsDetails']]
exp = new_df.explode('medicationsDetails')
exp.reset_index()
# exp = exp['medicationsDetails'].apply(pd.json_normalize)
# exp.keys


,id,firstName,lastName,medicationsDetails
0,L1rgGwC11HM8IuwTOdIY,Jane,Doe,"{'medName': 'b', 'selfAdmin': 'b', 'dosageFreq..."
1,L1rgGwC11HM8IuwTOdIY,Jane,Doe,"{'medName': 'a', 'selfAdmin': 'a', 'dosageFreq..."
2,k0Dlvf2amfTUM3ejugnJ,Alina,Chen,"{'medName': 'a', 'selfAdmin': 'a', 'dosageFreq..."
3,k0Dlvf2amfTUM3ejugnJ,Alina,Chen,"{'medName': 'a', 'selfAdmin': 'a', 'dosageFreq..."


In [91]:
detailInfo = pd.json_normalize(exp['medicationsDetails'])
detailInfo

,medName,selfAdmin,dosageFreq,admin,condition
0,b,b,b,b,b
1,a,a,a,a,a
2,a,a,a,a,a
3,a,a,a,a,a


In [92]:
exp = exp.drop('medicationsDetails', axis=1)

In [95]:
exp = exp.reset_index()

In [96]:
medDetails = pd.concat([exp, detailInfo], axis=1)
medDetails

,id,firstName,lastName,medName,selfAdmin,dosageFreq,admin,condition
0,L1rgGwC11HM8IuwTOdIY,Jane,Doe,b,b,b,b,b
1,L1rgGwC11HM8IuwTOdIY,Jane,Doe,a,a,a,a,a
2,k0Dlvf2amfTUM3ejugnJ,Alina,Chen,a,a,a,a,a
3,k0Dlvf2amfTUM3ejugnJ,Alina,Chen,a,a,a,a,a


In [43]:
new_df['medicationsDetails'][0]

[{'medName': 'b',
  'selfAdmin': 'b',
  'dosageFreq': 'b',
  'admin': 'b',
  'condition': 'b'},
 {'medName': 'a',
  'selfAdmin': 'a',
  'dosageFreq': 'a',
  'admin': 'a',
  'condition': 'a'}]

In [25]:
# pd.json_normalize(df, record_path='medicationsDetails', meta=[['firstName', 'lastName']])

# normCol = df['medicationsDetails'].apply(pd.json_normalize)

df['medicationsDetails'].apply(type)

id
3N7cAZgcP2mPY4I1LeGn    <class 'float'>
94uaRdZ9gUbrEuuBpqg3    <class 'float'>
L1rgGwC11HM8IuwTOdIY     <class 'list'>
XzkoIkEs6440B0zCB17G    <class 'float'>
haItGXi74WmDJ9rKv8Cu    <class 'float'>
k0Dlvf2amfTUM3ejugnJ     <class 'list'>
vp9L1YxDmkx8rSYOw2q6    <class 'float'>
Name: medicationsDetails, dtype: object

In [20]:
# print(type(df['medicationsDetails']))
import ast

df['medicationsDetails'] = df['medicationsDetails'].apply(ast.literal_eval)

df

ValueError: malformed node or string: nan

In [ ]:
# maybe set default value to null so that it's not just empty 
# separate medication table where each row = one person's medication

In [ ]:
pd.DataFrame.to_csv(df)